# Data Fusion Contest. Goodsification

# Часть 2

В этой части с помощью ранее обученной модели LinearSVC мы сделаем прогноз категории чека для всего исходного датасета, разобьем категории товаров на укрупненные группы и сгенерируем признак "тип чека" (т.е. наиболее популярная группа товаров в чеке)

### Импортируем необходимые библиотеки

In [60]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, GridSearchCV, GroupShuffleSplit, train_test_split, GroupKFold,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler


from nltk.corpus import stopwords 

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
stopwords = set(STOPWORDS)

from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import itertools

import scipy
from scipy.sparse import coo_matrix, hstack
import pickle
import re

from dask import dataframe as dd 
import dask.array as da

### Загружаем ранее обученную модель

In [47]:
import pickle
tfidf = pickle.load(open('tfidf', 'rb'))
clf = pickle.load(open('clf_task1', 'rb'))
scl = pickle.load(open('scl', 'rb'))
model = Word2Vec.load("word2vec.model")

### Загружаем и обрабатываем данные (аналогично с частью 1 для сокращенного датасета)

In [48]:
data_path='F:\\Data_Science\\competitions\\Data Fusion Contest VTB\\data'
data= pd.read_parquet(data_path+'\\train.parquet')

In [49]:
data.fillna('', inplace=True)

In [50]:
data.head()

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
0,7,5,09:50,Флуконазол капс 150мг №1 Вертекс,1.0,7,-1,-1,вертекс
1,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,
2,39,4,11:28,"Борщ Станичный с тушенкой, 103 ккал (завод, з/у)",1.0,6,1,-1,
3,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,


In [57]:
def clean_text(df):
    df["item_name"] = df["item_name"].apply(lambda x: x.lower())

    for symbol in [",", "%", "(", ")", "." , "\'", "/", "-", "_", "/", "\""]:
        df["item_name"] = df["item_name"].apply(lambda x: x.replace(symbol, " "))

    df["item_name"] = df["item_name"].apply(lambda x: x.replace("ржано", "ржаной "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("нач ", "начинкой "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace(" нач", " начинкой "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("orbit", " орбит "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("жев рез", " жевательная резинка "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("ж р ", " жевательная резинка "))
    
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("\"", " \" "))
    
    #v2
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("0 75вин", " вино "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace(" рыбн ", " рыбная "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("морепродуктами", " морепродукты "))
    
    #v3
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("snickers", " сникерс "))
    
    #v4
    df["item_name"] = df["item_name"].apply(lambda x: x.replace(" kуриная ", " куриная "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace(" kотлета ", " котлета "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace(" kофе ", " кофе "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("kофе ", "кофе "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace(" kофе", " кофе"))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("kофе", "кофе"))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace(" kаппучино ", " каппучино "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("сапоги", " обувь "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("туфли", " обувь "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("полусапоги", " обувь "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("кеды высокие", " обувь "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("кеды мужские", " обувь "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("кеды женские", " обувь "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("кеды детские", " обувь "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("кеды муж", " обувь "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("сандалии", " обувь "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("пижама", " пижам "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("пижамный", " пижам "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("пижамные", " пижам "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("для сна", " пижам "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("сорочка дет", " пижам "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("сорочка женская", " пижам "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("сорочка жен", " пижам "))
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("сорочка подростковая", " пижам ")) 
    df["item_name"] = df["item_name"].apply(lambda x: x.replace("kомплект", " комплект "))

    #v5
    df['item_name'] = df['item_name'].str.replace('№', '')
    df['item_name'] = df['item_name'].str.replace('%', '')
    df['item_name'] = df['item_name'].str.replace('╣', '')
    df['item_name'] = df['item_name'].str.replace('"', '')
    df['item_name'] = df['item_name'].str.replace('<', '')
    df['item_name'] = df['item_name'].str.replace('>', '')
    df['item_name'] = df['item_name'].str.replace('#', '')
    df['item_name'] = df['item_name'].str.replace('_', '')
    df['item_name'] = df['item_name'].str.replace('+', '')
    df['item_name'] = df['item_name'].str.replace('-', '')
    df['item_name'] = df['item_name'].str.replace('&', '')
    df['item_name'] = df['item_name'].str.replace('  ', ' ')
    df['item_name'] = df['item_name'].str.replace('a', 'а')
    df['item_name'] = df['item_name'].str.replace('h', 'н')
    df['item_name'] = df['item_name'].str.replace('k', 'к')
    df['item_name'] = df['item_name'].str.replace('b', 'в')
    df['item_name'] = df['item_name'].str.replace('c', 'с')
    df['item_name'] = df['item_name'].str.replace('o', 'о')
    df['item_name'] = df['item_name'].str.replace('p', 'р')
    df['item_name'] = df['item_name'].str.replace('t', 'т')
    df['item_name'] = df['item_name'].str.replace('x', 'х')
    df['item_name'] = df['item_name'].str.replace('y', 'у')
    df['item_name'] = df['item_name'].str.replace('e', 'е')
    df['item_name'] = df['item_name'].str.replace('m', 'м')
    df['item_name'] = df['item_name'].str.replace('  ', ' ')

    df['item_name'] = df['item_name'].str.lstrip()
    
    
clean_text(data)

data["item_name"]

0                             флуконазол капс 150мг 1 вертекс
1                                               молоко 3 2 шт
2               борщ станичный с тушенкой 103 ккал завод з у 
3                                    компот из изюма 114 ккал
4           макаронные изделия отварные масло сливочное 24...
                                  ...                        
45729304            карам ломтик апел лимон грейфр 1кг яшкино
45729310             сметана 20 залесский фермер 200 г пл ст 
45729312                             слива черная пантера 1кг
45729314        хлеб заварной ржаной пшен 0 4г нарезка берез 
45729315                         яблоки дискавери 1кг молдова
Name: item_name, Length: 26094077, dtype: object

In [58]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'[\d.,]+|[A-Z][.A-Z]+\b\.*|\w+|\S')

data["tokens"] = data["item_name"].apply(lambda x: [e.lower() for e in tokenizer.tokenize(x)])

In [ ]:
data.to_csv('data.csv',encoding='utf-8-sig')

In [61]:
data=pd.read_csv('data.csv', index_col=0, encoding='utf-8-sig')
data.fillna('', inplace=True);

F:\Data_Science\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
import numpy as np

def get_sentence_vector(model, words_list):
    size = len(words_list)
    sentence_vector = np.zeros(100)
    
    for word in words_list:
        try:
            sentence_vector += model.wv[word]
        except Exception:
            size -= 1
            
    if size > 0:
        sentence_vector /= size
        
    return sentence_vector

### Определение типа чека (наиболее популярная в чеке группа товаров)

Данные имеют очень большой объем, может возникнуть нехватка оперативной памяти. Поэтому будем прогнозировать данные по частям, а также использовать библиотеку dask.

### 1 часть

In [16]:
data_1=data[['receipt_id','category_id','item_name','tokens']].iloc[:8999999]

In [17]:
data_1

,receipt_id,category_id,item_name,tokens
0,7,-1,флуконазол капс 150мг 1 вертекс,"[флуконазол, капс, 150, мг, 1, вертекс]"
1,11,78,молоко 3 2 шт,"[молоко, 3, 2, шт]"
2,39,-1,борщ станичный с тушенкой 103 ккал завод з у,"[борщ, станичный, с, тушенкой, 103, ккал, заво..."
3,39,71,компот из изюма 114 ккал,"[компот, из, изюма, 114, ккал]"
4,39,71,макаронные изделия отварные масло сливочное 24...,"[макаронные, изделия, отварные, масло, сливочн..."
...,...,...,...,...
15776979,4526102,-1,перец фаршированный мясом и ри,"[перец, фаршированный, мясом, и, ри]"
15776984,4526102,-1,фарш домашний вес,"[фарш, домашний, вес]"
15776985,4526102,-1,хворост медовый 200г россия шт,"[хворост, медовый, 200, г, россия, шт]"
15776986,4526102,-1,чист ср во доместос роwеr 5 ле,"[чист, ср, во, доместос, роwеr, 5, ле]"


In [18]:
%%time
#применяем ранее обученный word2vec

model = Word2Vec.load("word2vec.model")

X_add2_1 = pd.DataFrame(np.array([get_sentence_vector(model, tokens) for tokens in data_1["tokens"]]))
X_add2_1 = scipy.sparse.csr_matrix(scl.transform(X_add2_1))
X_add2_1

Wall time: 3min 18s


<8999999x100 sparse matrix of type '<class 'numpy.float64'>'
	with 899974217 stored elements in Compressed Sparse Row format>

In [19]:
X_add1_1 = tfidf.transform(data_1.item_name)

In [20]:
X_add_1 = hstack([X_add1_1, scipy.sparse.csr_matrix(X_add2_1)])

In [21]:
pred_category_add_1 = clf.predict(X_add_1)

In [22]:
data_1['pred_category']=pred_category_add_1
data_1['pred_category']=data_1['pred_category'].apply(int)

In [23]:
data_1.to_csv('data_1.csv',encoding='utf-8-sig')

In [24]:
data_1

,receipt_id,category_id,item_name,tokens,pred_category
0,7,-1,флуконазол капс 150мг 1 вертекс,"[флуконазол, капс, 150, мг, 1, вертекс]",38
1,11,78,молоко 3 2 шт,"[молоко, 3, 2, шт]",78
2,39,-1,борщ станичный с тушенкой 103 ккал завод з у,"[борщ, станичный, с, тушенкой, 103, ккал, заво...",71
3,39,71,компот из изюма 114 ккал,"[компот, из, изюма, 114, ккал]",71
4,39,71,макаронные изделия отварные масло сливочное 24...,"[макаронные, изделия, отварные, масло, сливочн...",71
...,...,...,...,...,...
15776979,4526102,-1,перец фаршированный мясом и ри,"[перец, фаршированный, мясом, и, ри]",71
15776984,4526102,-1,фарш домашний вес,"[фарш, домашний, вес]",79
15776985,4526102,-1,хворост медовый 200г россия шт,"[хворост, медовый, 200, г, россия, шт]",84
15776986,4526102,-1,чист ср во доместос роwеr 5 ле,"[чист, ср, во, доместос, роwеr, 5, ле]",117


In [25]:
data_1.to_csv('data_1.csv',encoding='utf-8-sig')

### 2 часть

In [6]:
data_2=data[['receipt_id','category_id','item_name','tokens']].iloc[8999999:17999998]

In [7]:
data_2

,receipt_id,category_id,item_name,tokens
15776988,4526105,-1,анестезия,['анестезия']
15776989,4526105,-1,анестезия,['анестезия']
15776990,4526105,-1,анестезия инъекция,"['анестезия', 'инъекция']"
15776991,4526105,-1,пломбирование корневых каналов при лечении осл...,"['пломбирование', 'корневых', 'каналов', 'при'..."
15776992,4526105,-1,пломбировка коронки 1 кл жевательного зуба,"['пломбировка', 'коронки', '1', 'кл', 'жевател..."
...,...,...,...,...
31551707,9002832,-1,пакет,['пакет']
31551710,9002841,-1,колг жен s хl,"['колг', 'жен', 's', 'хl']"
31551713,9002846,-1,чай сurтis багама найтс 1 7гх20пир,"['чай', 'сurтis', 'багама', 'найтс', '1', '7',..."
31551716,9002847,-1,гб пельмени с маслом,"['гб', 'пельмени', 'с', 'маслом']"


In [8]:
X_add2_2 = pd.DataFrame(np.array([get_sentence_vector(model, tokens) for tokens in data_2["tokens"]]))
X_add2_2 = scipy.sparse.csr_matrix(scl.transform(X_add2_2))
X_add2_2

<8999999x100 sparse matrix of type '<class 'numpy.float64'>'
	with 899999900 stored elements in Compressed Sparse Row format>

In [20]:
data_2.fillna('',inplace=True)

In [21]:
X_add1_2 = tfidf.transform(data_2.item_name)
X_add_2 = hstack([X_add1_2, scipy.sparse.csr_matrix(X_add2_2)])
pred_category_add_2 = clf.predict(X_add_2)
data_2['pred_category']=pred_category_add_2
data_2['pred_category']=data_2['pred_category'].apply(int)

In [22]:
data_2

,receipt_id,category_id,item_name,tokens,pred_category
15776988,4526105,-1,анестезия,['анестезия'],45
15776989,4526105,-1,анестезия,['анестезия'],45
15776990,4526105,-1,анестезия инъекция,"['анестезия', 'инъекция']",38
15776991,4526105,-1,пломбирование корневых каналов при лечении осл...,"['пломбирование', 'корневых', 'каналов', 'при'...",45
15776992,4526105,-1,пломбировка коронки 1 кл жевательного зуба,"['пломбировка', 'коронки', '1', 'кл', 'жевател...",73
...,...,...,...,...,...
31551707,9002832,-1,пакет,['пакет'],203
31551710,9002841,-1,колг жен s хl,"['колг', 'жен', 's', 'хl']",61
31551713,9002846,-1,чай сurтis багама найтс 1 7гх20пир,"['чай', 'сurтis', 'багама', 'найтс', '1', '7',...",85
31551716,9002847,-1,гб пельмени с маслом,"['гб', 'пельмени', 'с', 'маслом']",73


In [23]:
data_2.to_csv('data_2.csv',encoding='utf-8-sig')

### 3 часть

In [24]:
data_3=data[['receipt_id','category_id','item_name','tokens']].iloc[17999998:]

In [25]:
X_add2_3 = pd.DataFrame(np.array([get_sentence_vector(model, tokens) for tokens in data_3["tokens"]]))
X_add2_3 = scipy.sparse.csr_matrix(scl.transform(X_add2_3))
X_add2_3

<8094079x100 sparse matrix of type '<class 'numpy.float64'>'
	with 809407900 stored elements in Compressed Sparse Row format>

In [26]:
data_3.fillna('',inplace=True)

In [27]:
X_add1_3 = tfidf.transform(data_3.item_name)
X_add_3 = hstack([X_add1_3, scipy.sparse.csr_matrix(X_add2_3)])
pred_category_add_3 = clf.predict(X_add_3)
data_3['pred_category']=pred_category_add_3
data_3['pred_category']=data_3['pred_category'].apply(int)

In [28]:
data_3

,receipt_id,category_id,item_name,tokens,pred_category
31551718,9002848,38,!перекись водорода р р д мест и нар прим я,"['!', 'перекись', 'водорода', 'р', 'р', 'д', '...",38
31551719,9002848,203,пакет,['пакет'],203
31551720,9002848,-1,бриллиантовый зеленый р р сп 1 10мл фл с,"['бриллиантовый', 'зеленый', 'р', 'р', 'сп', '...",70
31551721,9002848,-1,клотримазол акрихин мазь д наружн пр,"['клотримазол', 'акрихин', 'мазь', 'д', 'наруж...",38
31551723,9002878,84,батон нарезной из муки в с нарезка 380г русски...,"['батон', 'нарезной', 'из', 'муки', 'в', 'с', ...",84
...,...,...,...,...,...
45729304,10016159,-1,карам ломтик апел лимон грейфр 1кг яшкино,"['карам', 'ломтик', 'апел', 'лимон', 'грейфр',...",73
45729310,10016159,-1,сметана 20 залесский фермер 200 г пл ст,"['сметана', '20', 'залесский', 'фермер', '200'...",78
45729312,10016159,-1,слива черная пантера 1кг,"['слива', 'черная', 'пантера', '1', 'кг']",38
45729314,10016159,-1,хлеб заварной ржаной пшен 0 4г нарезка берез,"['хлеб', 'заварной', 'ржаной', 'пшен', '0', '4...",84


In [29]:
data_3.to_csv('data_3.csv',encoding='utf-8-sig')

### Объединяем все части и агрегируем данные по чекам

In [30]:
data_1=pd.read_csv('data_1.csv', index_col=0, encoding='utf-8-sig')
data_2=pd.read_csv('data_2.csv', index_col=0, encoding='utf-8-sig')
data_3=pd.read_csv('data_3.csv', index_col=0, encoding='utf-8-sig')

F:\Data_Science\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [32]:
data_total=pd.concat([data_1,data_2,data_3])

In [33]:
data_total

,receipt_id,category_id,item_name,tokens,pred_category
0,7,-1,флуконазол капс 150мг 1 вертекс,"['флуконазол', 'капс', '150', 'мг', '1', 'верт...",38
1,11,78,молоко 3 2 шт,"['молоко', '3', '2', 'шт']",78
2,39,-1,борщ станичный с тушенкой 103 ккал завод з у,"['борщ', 'станичный', 'с', 'тушенкой', '103', ...",71
3,39,71,компот из изюма 114 ккал,"['компот', 'из', 'изюма', '114', 'ккал']",71
4,39,71,макаронные изделия отварные масло сливочное 24...,"['макаронные', 'изделия', 'отварные', 'масло',...",71
...,...,...,...,...,...
45729304,10016159,-1,карам ломтик апел лимон грейфр 1кг яшкино,"['карам', 'ломтик', 'апел', 'лимон', 'грейфр',...",73
45729310,10016159,-1,сметана 20 залесский фермер 200 г пл ст,"['сметана', '20', 'залесский', 'фермер', '200'...",78
45729312,10016159,-1,слива черная пантера 1кг,"['слива', 'черная', 'пантера', '1', 'кг']",38
45729314,10016159,-1,хлеб заварной ржаной пшен 0 4г нарезка берез,"['хлеб', 'заварной', 'ржаной', 'пшен', '0', '4...",84


Мы получили прогноз категории товара для всего датасета. Теперь разобьем эти категории на группы:

In [34]:
'''
0 - алкоголь, презервативы, сигареты
1 - автомобильные принадлежности,бензинб услуги по автомобилям
2 - печатная продукция, канцтовары
3 - аптека, оптика, эфирные маслаб медицинские услуги
4 - бытовая химия, средства гигиеныб косметика
5 - одежда/обувь
6 - кафе/рестораны/фастфуд
7 - продуктовые магазины
8 - развлечения
9 - коммунальные услуги
10 - спорт
11 - ремонт/стройка
12 - для дома (типа hoff.)
13 - сад/огород
14 - для животных
15 - творчество
16 - провода/электроника
17 - другое

'''

dict_cat={0:0,  1:0,  2:0,  3:0, 
 4:1,  6:1,  7:1,  9:1,  11:1,  12:1,  13:1,
 19:2,  20:2,  24:2,  26:2,  27:2,  29:2,  30:2,  31:2,
 35:3,  36:3,  37:3,  38:3,  39:3,
 40:4,  41:4,  42:4,
 43:3,  45:3,
 46:4,  49:4,  50:4,  51:4,
 52:5,  53:5,  54:5,  55:5,  56:5,  57:5,  58:5,  60:5,  61:5,  62:5,
 66:6,  67:6,  68:6,  69:6,  70:6,  71:6,
 72:7,  73:7,  74:7,  75:7,  76:7,  77:7,  78:7,  79:7,  80:7,  81:7,  82:7,  83:7,  84:7,  85:7, 
 90:8,
 92:9,
 96:10,  97:10,  100:10,  101:10,  102:10,  103:10,
 105:11,  106:11,  107:11,  108:11,  109:11, 111:11,   114:11,  115:11,
 117:12,  118:12,  120:12,  128:12,  130:12,  133:12,
 138:13,  139:13,  140:13,
 145:14,  150:14,
 163:15,  164:15,  167:15,
 177:16,
 203:17,  204:17
   
}

In [35]:
def item_type(x):
  return dict_cat[x]

In [36]:
def most_popular(arr):
  return np.argmax(np.bincount(arr))

Дальнейшие операции в pandas не пошли. Зависло все намертво. Поэтому используем dask.

In [37]:
data_total_dask= dd.from_pandas(data_total,npartitions=3)

In [38]:
data_total_dask['item_type']=data_total_dask['pred_category'].apply(item_type)

F:\Data_Science\Anaconda\lib\site-packages\dask\dataframe\core.py:3358: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('pred_category', 'int64'))

  warnings.warn(meta_warning(meta))


In [39]:
data_total_dask=data_total_dask.groupby('receipt_id')['item_type'].apply(most_popular).reset_index()

<ipython-input-39-487d9ac3809f>:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  data_total_dask=data_total_dask.groupby('receipt_id')['item_type'].apply(most_popular).reset_index()


In [40]:
data_check_type=data_total_dask.compute()
data_check_type.rename(columns={'item_type': 'receipt_type'}, inplace=True)
data_check_type

,receipt_id,receipt_type
0,6,17
1,7,3
2,14,4
3,16,7
4,17,6
...,...,...
2707512,10016146,7
2707513,10016149,6
2707514,10016151,7
2707515,10016157,6


In [41]:
data_check_type.to_csv('data_check_type.csv',encoding='utf-8-sig')

In [54]:
train=pd.read_csv('train_without_receipt_type.csv', index_col=0, encoding='utf-8-sig')

In [55]:
train=train.merge(data_check_type[['receipt_id','receipt_type']], on='receipt_id')

In [44]:
train.to_csv('train_with_receipt_type.csv',encoding='utf-8-sig')

Т.о. в этой части мы сгенерировали новый признак для итоговой модели: тип чека. Надеемся, что это поможет нам в классификации трудноразделимых наименований (например, "куриное филе" в ресторане и "куриное филе на полке магазина"). Окончательная модель catboost - в третей части решения.